In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [2]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
# NETWORK = 'EfficientNetB7'
# NETWORK = 'NASNetLarge'
NETWORK = 'ConvNeXtBase'

METHOD = "AVG-Max"

In [3]:
WORKING_DIRECTORY = '/home/amirhosein/HECKTOR2022/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/home/amirhosein/HECKTOR2022/hecktor2022_patient_endpoint_training.csv"

In [4]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,CHUM-001,0.382542,0.482160,0.583147,1.196617,0.624400,0.586671,0.112734,0.374602,-0.137221,...,0.049612,0.535139,0.330014,0.832704,0.011301,-0.070633,0.396939,0.839534,0.659166,0.425448
1,CHUM-002,0.216190,0.524983,0.551176,0.821671,0.445858,0.569002,-0.091425,0.468208,-0.217389,...,0.303546,0.673573,0.430209,0.960595,0.076832,-0.021528,0.554834,0.449408,0.496389,0.245455
2,CHUM-006,-0.395082,0.624427,0.685935,1.439608,0.473126,0.206268,-0.088705,0.226031,0.079891,...,0.004275,0.666356,0.549611,0.958323,-0.257578,0.129374,0.380883,0.243307,-0.302199,0.426647
3,CHUM-007,-0.247301,0.529179,0.648351,1.495837,0.471568,0.402793,0.068450,0.166422,0.056842,...,0.137718,0.800176,0.608315,0.969192,-0.253268,-0.111630,0.322875,0.256930,0.110295,0.274856
4,CHUM-008,0.012528,0.726519,0.592451,1.611256,0.479185,0.595648,-0.092058,0.389724,-0.040278,...,-0.026297,0.769565,0.275308,0.931881,-0.158165,0.041871,0.338004,0.079147,-0.540295,0.148190


# Up to here

Read Outcome csv

In [5]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [6]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [7]:
OD.columns[0]

'Patient_ID'

In [8]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [9]:
OD.columns[0]


'Patient_ID'

In [10]:
PET_outcome_data.columns[0]

'Patient_ID'

In [11]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,CHUM-001,0.382542,0.482160,0.583147,1.196617,0.624400,0.586671,0.112734,0.374602,-0.137221,...,0.049612,0.535139,0.330014,0.832704,0.011301,-0.070633,0.396939,0.839534,0.659166,0.425448
1,CHUM-002,0.216190,0.524983,0.551176,0.821671,0.445858,0.569002,-0.091425,0.468208,-0.217389,...,0.303546,0.673573,0.430209,0.960595,0.076832,-0.021528,0.554834,0.449408,0.496389,0.245455
2,CHUM-006,-0.395082,0.624427,0.685935,1.439608,0.473126,0.206268,-0.088705,0.226031,0.079891,...,0.004275,0.666356,0.549611,0.958323,-0.257578,0.129374,0.380883,0.243307,-0.302199,0.426647
3,CHUM-007,-0.247301,0.529179,0.648351,1.495837,0.471568,0.402793,0.068450,0.166422,0.056842,...,0.137718,0.800176,0.608315,0.969192,-0.253268,-0.111630,0.322875,0.256930,0.110295,0.274856
4,CHUM-008,0.012528,0.726519,0.592451,1.611256,0.479185,0.595648,-0.092058,0.389724,-0.040278,...,-0.026297,0.769565,0.275308,0.931881,-0.158165,0.041871,0.338004,0.079147,-0.540295,0.148190


In [12]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [13]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [14]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1016,1017,1018,1019,1020,1021,1022,1023,Relapse,RFS
0,CHUM-001,0.382542,0.482160,0.583147,1.196617,0.624400,0.586671,0.112734,0.374602,-0.137221,...,0.330014,0.832704,0.011301,-0.070633,0.396939,0.839534,0.659166,0.425448,0,1704
1,CHUM-002,0.216190,0.524983,0.551176,0.821671,0.445858,0.569002,-0.091425,0.468208,-0.217389,...,0.430209,0.960595,0.076832,-0.021528,0.554834,0.449408,0.496389,0.245455,1,439
2,CHUM-006,-0.395082,0.624427,0.685935,1.439608,0.473126,0.206268,-0.088705,0.226031,0.079891,...,0.549611,0.958323,-0.257578,0.129374,0.380883,0.243307,-0.302199,0.426647,0,1186
3,CHUM-007,-0.247301,0.529179,0.648351,1.495837,0.471568,0.402793,0.068450,0.166422,0.056842,...,0.608315,0.969192,-0.253268,-0.111630,0.322875,0.256930,0.110295,0.274856,0,1702
4,CHUM-008,0.012528,0.726519,0.592451,1.611256,0.479185,0.595648,-0.092058,0.389724,-0.040278,...,0.275308,0.931881,-0.158165,0.041871,0.338004,0.079147,-0.540295,0.148190,0,1499


In [15]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)